In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
%cd /content/drive/MyDrive/Neuron\ Segmentation\ Research\ 2023

/content/drive/MyDrive/Neuron Segmentation Research 2023


In [9]:
!git init neuron_segmentation

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/drive/MyDrive/Neuron Segmentation Research 2023/neuron_segmentation/.git/


In [10]:
%cd neuron_segmentation/

/content/drive/MyDrive/Neuron Segmentation Research 2023/neuron_segmentation


In [11]:
%ls -a

.git/


In [12]:
!git status

On branch master

No commits yet

nothing to commit (create/copy files and use "git add" to track)


dataset/  .git/  .ipynb_checkpoints/
